#### Description

This code uses <b>Gradient Descent</b> to find optimal hyperparameters for the GA_2_2 GeneticAlgorithm run method:
* chromosomes
* islands
* num_parents
* gene_flow_rate

From GA_Testing_2_2_hp and GA_Testing_2_2_mutation_rate_2, we see that base_mutation_rate should be around 0.05. Here we fix it as a control variable and vary the above values. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from GA_2_2 import GeneticAlgorithm

In [2]:
# Control variables
cnn_model_path = '../Models/CNN_6_1_2.keras'
masked_sequence = 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'
max_length = 150
pop_size = 200
generations = 100
precision = 0.1
base_mutation_rate = 0.05

In [3]:
def objective_function(chromosomes, islands, num_parents, gene_flow_rate, target_expression):
    errors = []
    for run_id in range(run_per_combination):
        print(f'Run {run_id + 1}/{run_per_combination} for combination {chromosomes}, {islands}, {num_parents}, {gene_flow_rate}', end='\r')
        ga = GeneticAlgorithm(
            cnn_model_path=cnn_model_path,
            masked_sequence=masked_sequence,
            target_expression=target_expression,
            max_length=max_length,
            pop_size=pop_size,
            generations=generations,
            base_mutation_rate=base_mutation_rate,
            precision=precision,
            chromosomes=chromosomes,
            islands=islands,
            num_parents=num_parents,
            gene_flow_rate=gene_flow_rate,
            print_progress=False
        )
        try:
            best_sequence, best_prediction = ga.run()
            errors.append(abs(best_prediction - target_expression))
        except Exception as e:
            errors.append(float('inf'))  # Penalize failures
    return np.mean(np.square(errors))  # MSE

def compute_gradients(params, target_expression, epsilon=1e-3):
    gradients = {}
    base_error = objective_function(*params, target_expression)
    for i, key in enumerate(['chromosomes', 'islands', 'num_parents', 'gene_flow_rate']):
        params_copy = list(params)
        if key != 'gene_flow_rate':
            params_copy[i] += 1
        else:
            params_copy[i] += epsilon
        gradients[key] = (objective_function(*params_copy, target_expression) - base_error) / (1 if key != 'gene_flow_rate' else epsilon)
    return gradients


def update_parameters(params, gradients, learning_rate=0.1):
    updated_params = []
    for i, key in enumerate(['chromosomes', 'islands', 'num_parents', 'gene_flow_rate']):
        param = params[i] - learning_rate * gradients[key]
        if key != 'gene_flow_rate':
            param = max(1, round(param))
        updated_params.append(param)
    return updated_params

In [ ]:
params = [8, 8, 8, 0.05]  # Initial guess
learning_rate = 0.1
gradient_tolerance = 1e-6
error_tolerance = 1e-3
max_iterations = 100
run_per_combination = 5
target_expression = 1
results = []

for iteration in range(max_iterations):
    print(f"Iteration {iteration + 1} | Params: {params}")

    gradients = compute_gradients(params, target_expression)

    max_gradient_magnitude = max(abs(g) for g in gradients.values())
    if max_gradient_magnitude < gradient_tolerance:
        print(f"Gradients below tolerance: {max_gradient_magnitude:.6f}")
        break

    params = update_parameters(params, gradients, learning_rate)
    error = objective_function(*params, target_expression)

    print(f"\nError: {error:.4f}\n")
    
    # Append results for the current iteration
    results.append({
        'iteration': iteration + 1,
        'chromosomes': params[0],
        'islands': params[1],
        'num_parents': params[2],
        'gene_flow_rate': params[3],
        'error': error
    })

    if error < error_tolerance:
        print(f"Error below tolerance: {error:.5f}")
        break

Iteration 1 | Params: [8, 8, 8, 0.05]
Instructions for updating:
Use tf.identity with explicit device placement instead.


c:\Users\rsore\anaconda3\envs\TX_prediction\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Error: 0.0034ombination 8, 8, 8, 0.057568656063057214

Iteration 2 | Params: [8, 8, 8, 0.05756865606305721]
Error: 0.0032ombination 8, 8, 8, 0.14016577937273951

Iteration 3 | Params: [8, 8, 8, 0.1401657793727395]
Error: 0.0057ombination 8, 8, 8, 0.22486610527541298

Iteration 4 | Params: [8, 8, 8, 0.22486610527541298]
Error: 0.0036ombination 8, 8, 8, 0.17699349868960048

Iteration 5 | Params: [8, 8, 8, 0.1769934986896004]


KeyboardInterrupt: 

In [ ]:
print(f"Final parameters: {params}, Error: {error:.4f}")

In [ ]:
results_df = pd.DataFrame(results)
results_df.sort_values(by='average_error').head()

In [ ]:
results_df.to_csv('../Data/Results/GA_Testing_2_2_hp_3.csv', index=False)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(25, 12), sharey=True)
dependent_variables = ['chromosomes', 'islands', 'num_parents', 'gene_flow_rate']
polynomial_degree = 2

for i, dependent_variable in enumerate(dependent_variables):
    ax = axs[i]
    ax.scatter(results_df[dependent_variable], results_df['average_error'], s=30, alpha=0.7)

    # Fit and plot a polynomial regression line
    poly_fit = np.polyfit(results_df[dependent_variable], results_df['average_error'], polynomial_degree)
    poly_fn = np.poly1d(poly_fit)
    x_vals = np.linspace(results_df[dependent_variable].min(), results_df[dependent_variable].max(), 100)
    ax.plot(x_vals, poly_fn(x_vals), color='red')

    # Set subplot title and labels
    ax.set_title(f'{dependent_variable} vs average_error')
    ax.set_xlabel(dependent_variable)
    if i == 0:
        ax.set_ylabel('average_error')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Visualizing gradient descent algorithm

# Extracting data
iterations = [result['iteration'] for result in results]
errors = [result['error'] for result in results]
gradient_magnitudes = []

for result in results:
    gradients = compute_gradients([result['chromosomes'], result['islands'], result['num_parents'], result['gene_flow_rate']], target_expression)
    gradient_magnitude = np.sqrt(sum(g**2 for g in gradients.values()))
    gradient_magnitudes.append(gradient_magnitude)

# Combined Plot
fig, axs = plt.subplots(3, 1, figsize=(10, 18))

# Error vs Iterations
axs[0].plot(iterations, errors, marker='o', color='blue')
axs[0].set_title("Error vs Iterations")
axs[0].set_xlabel("Iteration")
axs[0].set_ylabel("Error")
axs[0].grid()

# Parameter Values vs Iterations
for param in params:
    values = [result[param] for result in results]
    axs[1].plot(iterations, values, label=param)
axs[1].set_title("Parameter Values vs Iterations")
axs[1].set_xlabel("Iteration")
axs[1].set_ylabel("Parameter Values")
axs[1].legend()
axs[1].grid()

# Gradient Magnitude vs Iterations
axs[2].plot(iterations, gradient_magnitudes, marker='o', color='red')
axs[2].set_title("Gradient Magnitude vs Iterations")
axs[2].set_xlabel("Iteration")
axs[2].set_ylabel("Gradient Magnitude")
axs[2].grid()

plt.tight_layout()
plt.show()
